In the following Jupyter notebook, we will learn how to search for a particular process that occurred in a particle collider by scanning the invariant mass spectrum of many events detected in the Compact Muon Solenoid (CMS) detector. The process in question is a neutral $Z$ Boson ($Z^{0}$) decay to two leptons of the same flavor but opposite charge. That is, $Z \to l^{+}l^{-}$.

We will first import ROOT. ROOT is an analysis framework that's mostly used by scientists working on particle colliders. To learn more, please visit: https://root.cern/

In [1]:
try:
    import ROOT
except ImportError as E:
    print(f"Error importing ROOT: {E}")

ERROR in cling::CIFactory::createCI(): cannot extract standard library include paths!
Invoking:
  LC_ALL=C x86_64-conda-linux-gnu-c++   -DNDEBUG -xc++ -E -v /dev/null 2>&1 | sed -n -e '/^.include/,${' -e '/^ \/.*++/p' -e '}'
Results was:
With exit code 0


Welcome to JupyROOT 6.24/06


We now need to enable "jsroot", which of course stands for "JavaScript ROOT". This allows for us to turn our plot outputs into interactive environments. Now, every time we output a plot, we can explore it or configure it to our liking before deciding to produce a "final copy", for example.

In [2]:
%jsroot on

Next, we need to import the data that we wish to analyze. Since most collider data is stored in ".root" files, we needed a way to read the data from .root files. That is why we are using a PyROOT interface (because we know Python, not complicated C++ nonsense). Reading a .root file is as simple as reading any other file except that we need to use ROOT's method to read it. That is "TFile.Open()". All we require for this function is the hyperlink where the data is located.

In [3]:
LINK_OF_13GEV_DATA = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/1largeRjet1lep/MC/mc_361106.Zee.1largeRjet1lep.root"
datafile = ROOT.TFile.Open(LINK_OF_13GEV_DATA)

In order to plot in ROOT, it seems that we require instantiating a "Canvas" on which our plots will be drawn. The invariant mass spectrum usually is plotted as a histogram with some fixed bin width (what width we choose depends on the nature of our measurement -- it is usually not a trivial matter). Thus, we require first a Canvas on which to draw the histogram. Let's do that:

In [4]:
CANVAS_HEADER = "C"
HISTOGRAM_WIDTH = 800
HISTOGRAM_HEIGHT = 600
canvas = ROOT.TCanvas("Canvas", CANVAS_HEADER, HISTOGRAM_WIDTH, HISTOGRAM_HEIGHT)

Who knows what's going on in the step below, but perhaps it can be likened to reading a Pandas DataFrame. We read the data out to something called a "tree".

In [5]:
datatree = datafile.Get("mini")
print(f"We have {datatree.GetEntries()} entries in our tree.")

We have 53653 entries in our tree.


We now create a histogram with ROOT. As usual, we give it a title and define the range of the plot. Then, we assign a number of bins for our histogram.

In [6]:
HISTOGRAM_HEADER = "invariant_mass_histogram"
HISTOGRAM_TITLE = "Reconstructing the Z Boson Mass; invariant mass [GeV]; events [N]"
HISTOGRAM_NUMBER_OF_BINS = 30
HISTOGRAM_LOWER_RANGE = 40
HISTOGRAM_UPPER_RANGE = 140
invariant_mass_histogram = ROOT.TH1F(HISTOGRAM_HEADER, HISTOGRAM_TITLE, HISTOGRAM_NUMBER_OF_BINS, HISTOGRAM_LOWER_RANGE, HISTOGRAM_UPPER_RANGE)

Just as a check, since we have decided on a total of 30 histogram bins to use, and our plot range features masses in an interval of $140 GeV - 40 GeV = 100 GeV$, we each bin will span $100 GeV / 30$ bins $ \approx 3.33 GeV / $ bin.

Right now, our histogram has no data. All we have is its "exoskeleton". We now need to perform the steps that will fill the histogram with data. That involves the following: Our tree has data for each event that the collider detected. The data includes important quantites that each component of CMS measured. What quantities are important for our particular interaction? We are looking for two leptons of opposite charge but of the same generation. Those are the detections that we can use to infer the mass of the $Z^{0}$. 

Then, we construct two 4-vectors for each lepton in a pair. We are guided by this method because energy and momentum are conserved in an event. In order to determine what particle decayed to the two detected leptons, we must use the data we captured from their interaction with the detector. Only then may we begin to infer what particle they decayed from using conservation of energy and momentum. (Heavy, force-carrying Bosons decay very quickly if they are produced in collisions. So, the detector always captures information about the decay products of those heavier particles.) Let's set up the 4-vectors using ROOT's method "TLorentzVector()". This is an instantiation. We will fill it in with data later.

In [7]:
leadLepton = ROOT.TLorentzVector()
trailLepton = ROOT.TLorentzVector()

There is a bit more complicated physics that we must take into account in order to perform this analysis. Like energy and momentum conservation, there are other quantities that must be conserved in a particular event. In the decay channel that we are investigating, ($Z^{0} \to l^{+}l^{-}$), we notice that the leptons must be of the same family (that is why they are $\underline{both}$ written with $l$, and they must be oppositely charged (that's the $(+)$ and $(-)$). 

Of course, the CMS detector simply records at some time whatever particles happen to interact with it. So, we must be cautious about what events we use to reconstruct our particular interaction. We cannot hope to recover a $Z^{0}$ if we choose to analyze events where charge is not conserved. We also need to constrain our analysis to leptons of the same family (electrons or muons that are detected in CMS' electromagnetic calorimeter [ECAL]). That is what we will do next.

Please note that, in the data tree, "lep_charge" is a property that tells us whether or not that detected lepton is positively charged or negatively charged and "lep_type" refers to the family of lepton. The two if statements below correspond directly to conservation of charge and matching the family of the leptons.

Finally, when we have filtered out the irrelevant events in our data, we construct the 4-vectors for each of the leptons. This is performed by accessing more properties of the data tree. The 4-vector in question is what ROOT calls "PtEtaPhiE", which means that the four components of the vector are $(p_{T}, \eta, \phi, E)$. That's all jargon unless we understand the standard coordinate system that CMS uses. The fourth component of the 4-vector, $E$, of course, is the energy of the lepton. Since leptons carry charge, the ECAL can obtain their energy. The first component, $p_{T}$, is the transverse momentum of the lepton. That is, the component of the lepton's 3-momentum perpendicular to the beam direction. $\phi$ refers to the standard azimuthal angle. Finally, $\eta$ refers to what's called the $\textit{pseudorapidity}$ of the lepton. The pseudorapidity is defined to be $\eta = - ln \left[tan(\frac{\theta}{2})\right]$ (where $\theta$ actually is the polar angle in spherical polar coordinates), and is a way to measure a particle's angle with respect to the beam axis. These words won't make much sense without a proper diagram. I will refer to reader to Google for a better explanation of the geometry of CMS' standard coordinate system.

Anyway, the relevance of those quantities comes into play below when we access those properties using "lep_pt" (transverse momentum), "lep_eta" (pseudorapidity), "lep_phi" (azimuthal angle), "lep_E" (energy).

In [8]:
for event in datatree:
    if datatree.lep_n >= 2:
        if (datatree.lep_charge[0] != datatree.lep_charge[1]):
            if (datatree.lep_type[0] == datatree.lep_type[1]):
                    # We divide by 1000 to recover units of GeV.
                leadLepton.SetPtEtaPhiE(datatree.lep_pt[0] / 1000., datatree.lep_eta[0], datatree.lep_phi[0], datatree.lep_E[0] / 1000.)
                trailLepton.SetPtEtaPhiE(datatree.lep_pt[1] / 1000., datatree.lep_eta[1], datatree.lep_phi[1], datatree.lep_E[1] / 1000.)
                    # Next, we add the 4-vectors. This is employing conservation laws.
                invariant_mass = leadLepton + trailLepton
                invariant_mass_histogram.Fill(invariant_mass.M())

Finally, we do a bit of aesthetics:

In [9]:
invariant_mass_histogram.Draw()
invariant_mass_histogram.SetFillColor(4)

Now that the histogram actually has data, we can add the histogram to the Canvas and show the final product:

In [10]:
canvas.Draw()

Notice immediately that the width of the histogram bars do not always precisely land on the tick marks of the $x-$axis. What did we define the "bin-width" to be? Also, we notice that the number of entries (37415) does not match the original number of events that we had (53653). Clearly, those if statements filtered out events that did not follow our selection criteria. 